# ChatGPT를 활용한 자연어처리 자동화

대규모 언어 모델 (LLM)이 계속 커져나감에 따라, 웬만한 기업들조차 최첨단 언어 모델을 처음부터

학습하기도 무리인 동시에, 그 모델을 로컬에서 실행시킬 수 있는 환경이 안되는 경우가 태반입니다.

- ChatGPT의 경우 서비스만을 위해 사용되는 GPU가 (Nvidia A100) 2만개 이상으로 추산됨
- Meta와 X는 다음 세대 LLM을 위하여 10만개 이상의 GPU를 학습에 사용중에 있음


우리가 만약 최첨단 성능과 안정성을 바란다면, 비용을 지불하고 ChatGPT나 Claude이 제공하는 API를 통해

쿼리를 주고 받으면서 이에 대한 처리를 요청할 수 있습니다.

이 예제코드는 ChatGPT를 활용하여 과거의 뉴스 기사들을 일별로 요약 및 번역을 해봅니다.

# Requirements

Python 패키지
- pandas
- langchain
- langchain-openai
- python-dotenv

In [ ]:
!pip install pandas langchain langchain-openai python-dotenv gdown

# OpenAI API 발급받기 / 요금 충전 하기

OpenAI를 사용하기 위해서는 API를 발급하고 미리 크레딧을 충전해놓아야 합니다.

https://platform.openai.com/


# ChatGPT 불러오기

Langchain 라이브러리는 OpenAI API와 소통할 수 있는 간단한 클래스를 제공해줍니다.

다음은 간단하게 OpenAI의 `gpt-4o-mini`를 이용해보는 코드입니다

In [ ]:
import os

import dotenv
from langchain_openai import ChatOpenAI

# .env 파일을 생성하고 그 안에 
# OPENAI_API_KEY=sk-proj-어쩌구저쩌구API키
# 를 입력하거나 아래 os.environ 선언하는 것에 입력하세요.
dotenv.load_dotenv()

if ("OPENAI_API_KEY" not in os.environ) or ("sk-proj" not in os.environ["OPENAI_API_KEY"]):
    os.environ["OPENAI_API_KEY"] = "" # 여기에 발급받은 API 키를 입력하세요.


# invoke 등으로 호출 시 비용이 발생합니다.
llm = ChatOpenAI(
    temperature = 0.0, # 모델의 무작위성을 결정한다. 높을수록 더욱 창의적인 답변을 내놓는다.
    max_tokens=2048, # 모델이 생성할 수 있는 토큰 수의 제한을 정한다.
    model_name = "gpt-4o-mini", #사용할 모델의 종류를 정한다.
)

question = "대한민국을 한 문단 내로 간략히 설명해주세요."
result = llm.invoke(question)
#print(f"{result.content}")
# (colab용 보기좋게) 개행 삽입하기
print(".\n".join(f"{result.content}".split(".")))


반환된 결과 `result` 에는 입출력시 몇개의 토큰이 오갔는지 볼 수 있는 `.usage_metadata`가 있다.

In [ ]:
result.usage_metadata

# 뉴스 데이터 불러오기

우리는 과거 러시아-우크라이나 전쟁 뉴스를 가지고
- 하루단위로 있었던 해외 뉴스 기사 내 사건들을 요약하고
- 이를 한글로 번역해서 보여주도록

ChatGPT에게 지시를 해보는 예제를 만들어 볼 것입니다.

In [ ]:
# 데이터 다운로드받기
!mkdir data
!gdown https://drive.google.com/uc?id=11hMJSPTc7nlylk-xxR1ZSlxn2VMzEdYh -O ./data/Guardians_Russia_Ukraine.csv

In [ ]:
import pandas as pd

news_df = pd.read_csv("data/Guardians_Russia_Ukraine.csv")
news_df.head(5)

먼저, 데이터를 시간순으로 정렬합니다.

우리는 각 일별로 데이터를 보고 싶으므로, 년-월-일로 이루어진 필드를 새롭게 만들어 볼 것입니다.

In [ ]:
# 데이터 시간순 정렬 
news_df = news_df.sort_values(by="published")

# 데이터 필드 생성
news_df["date"] = news_df["published"].apply(lambda x: x[:10])

# 기사의 일별 날짜 종류 (중복 제거)
dates = news_df["date"].unique()

In [ ]:
# 살펴볼 날짜들
dates

# 입력 전처리하기

우리는 GPT에게 그 날에 있었던 기사들을 한번에 보내어 그 하루 내 있었던 이슈들을 요약하여 반환하게 하고 싶습니다.

먼저 일 단위로 하루에 있었던 모든 기사들을 일정한 형식으로 묶어내도록 전처리하겠습니다.

In [ ]:
# 입력받은 모든 기사를 하나의 문자열로 반환합니다.
def preprocessing(df):
    articles_str = ""
    for idx in df.index:
        articles_str += f"""
Title: {df.loc[idx, "headlines"]}
Body:
{df.loc[idx, "articles"]}\n
"""
    return articles_str

# 기사 몇개로 전처리를 테스트해봅시다.

tgt_date = "2022-02-24"
test_news_df = news_df[news_df["date"]==tgt_date]
print(preprocessing(test_news_df))

# 프롬프트와 파이프라인 설계 및 자동화 결과 얻기

앞에서 전처리하는 함수를 이용하여, 기사 내용을 넣어주면 그 날에 있었던 이슈들을 요약해서 한글로 알려주도록 GPT에게 질의하는 파이프라인을 설계해보겠습니다.

지시문를 잘 주어야 좋은 결과를 얻을 수 있습니다. 일단 간단한 지시문부터 시작해봅시다.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 지시문을 잘 주어야 합니다. 간단한 것으로 시작해봅시다.
template = """
다음은 우크라이나-러시아 전쟁 뉴스 기사들입니다. 그 기사들을 보고, 있었던 일들을 요약하고 한글로 번역해주세요.
###
{articles}
###
"""

# 위의 템플릿 문자열로 LangChain의 프롬프트 객체를 생성합니다.
prompt = PromptTemplate.from_template(template=template)

# LangChain에서 체인을 형성하는 방법입니다. | 으로 두 요소들을 연결합니다.
chain = prompt | llm | StrOutputParser()

In [ ]:
tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

출력 형식이 마음에 들지 않을 수 있습니다. 이럴 땐 지시문으로 출력을 구조화하도록 유도할 수 있습니다.

In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 이 때 사건 단위로 요약하여 다음처럼 나열해주세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

출력 형식
```
**요약:**
- 사건 1
- 사건 2
- 사건 3
- 사건 4
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

In [ ]:
tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

잘 나오는 것으로 보이니, 모든 날짜에 대해서 자동화를 해보겠습니다.

이 때 여러 입력을 묶어서 처리하고 싶을 때 `.batch()` 메소드를 사용할 수 있습니다.

`.batch()`는 여러 입력을 함께 묶어 OpenAI에 전달하며 좀 더 빠른 속도를 보여줍니다.


In [ ]:
articles_strs = []

for tgt_date in news_df["date"].unique():
    articles_strs.append(preprocessing(news_df[news_df["date"]==tgt_date]))
answers = chain.batch(inputs=articles_strs)

# 파일로 저장해봅니다.
with open("우크라이나-러시아 전쟁_요약본.txt","w") as file:
    for tgt_date, ans in zip(news_df["date"].unique(), answers):
        file.write(f"날짜: {tgt_date}\n{ans}\n")

# 이것저것 바꾸어보기

## 1. 서술식 문장 => 개조식 문장으로


In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 이때 사건 단위로 요약하여 다음처럼 나열해주세요.
- 서술식이 아닌 개조식으로 적어주세요.
  - 개조식 예시: 러시아는 미국과의 포로 교환을 논의할 준비가 되어있다고 밝힘
- 영어로 적지 말고 한글로 번역해서 적어주세요.

출력 형식
```
**요약:**
- 사건 1
- 사건 2
- 사건 3
- 사건 4
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

## 2. 각 뉴스 기사를 따로 요약하고, 마지막에 하루 전체를 요약하도록 하기

In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 기사 마다, 기사 제목으로 시작한 뒤 내용을 요약하세요
- 맨 마지막에 모든 기사의 사건들을 모아 요약해주세요
- 일어난 사건 단위로 요약하여 나열해주세요.
- 영어로 적지 말고 한글로 번역해서 적어주세요.

출력 형식
```
**요약:**
기사 1 제목: (기사 1의 제목입니다)
- (사건 1)
- (사건 2)
- (사건 3)

기사 2 제목: (기사 2의 제목입니다)
- (사건 1)
- (사건 2)

기사 3 제목: (기사 3의 제목입니다)
- (사건 1)
- (사건 2)
- (사건 3)
- (사건 4)

전체 요약:
- (사건 1)
- (사건 2)
- (사건 3)
- (사건 4)
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

In [ ]:
# 제목 검증해보기
for h in news_df[news_df["date"]==tgt_date]["headlines"]: print(h)

In [ ]:
# 날짜를 바꾸어서 한번 더
prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-08-01"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

In [ ]:
# 제목 검증해보기
for h in news_df[news_df["date"]==tgt_date]["headlines"]: print(h)

## 3. 전황과 인물들의 발언 모아보기


In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 전쟁 상황과 인물들의 발언을 따로 나누어서 제시하세요.
- 일어난 사건 단위로 요약하여 나열해주세요.
- 영어로 적지 말고 한글로 번역해서 적어주세요.

출력 형식
```
전쟁 상황
- 사건 1
- 사건 2
- 사건 3
- 사건 4

관련 발언
- 발언 1
- 발언 2
- 발언 3
- 발언 4
- 발언 5
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

## 4. 블로그 글 써보게 하기

In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 자유형식으로 2~3 문단 정도로 블로그 글을 써주세요.
- 정보를 알리려는 목적의 글이며, 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해야 합니다.
- 그 날에 올라온 기사들이다 라고 생각하고 블로그를 작성합니다.
- 길이는 최대 4문단 정도 되어야 합니다.
- 중대한 사건들은 강조해주세요.
- 독자들에게 친숙하게 구어체를 사용하세요.
- 영어로 적지 말고 한글로 번역해서 적어주세요.

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

## 5. 더 좋은 모델 써보기 (4번 블로그글 쓰기를 좋은 모델로)

In [ ]:
better_llm = ChatOpenAI(
    temperature = 0.0,
    max_tokens=2048,
    model_name = "gpt-4o"
)

template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 자유형식으로 2~3 문단 정도로 블로그 글을 써주세요.
- 정보를 알리려는 목적의 글이며, 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해야 합니다.
- 그 날에 올라온 기사들이다 라고 생각하고 블로그를 작성합니다.
- 길이는 최대 4문단 정도 되어야 합니다.
- 중대한 사건들은 강조해주세요.
- 독자들에게 친숙하게 구어체를 사용하세요.
- 영어로 적지 말고 한글로 번역해서 적어주세요.

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | better_llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)